In [5]:
import time
import datetime as dt
import pyautogui as gui
import numpy as np
import cv2
import pytesseract
import os
import re
import platform

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# -------------------- CONFIG --------------------
# Ajuste se o Tesseract não estiver no PATH (Windows):
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Pontos de clique (seu layout/zoom pode mudar!):
COORD = {
    "origem": (500, 400),      # campo origem
    "destino": (813, 400),     # campo destino
    "data_ida": (820, 400),    # campo data ida
    "data_volta": (1020, 400), # campo data volta
    "buscar": (1200, 900),     # botão buscar
    # Posição onde aparece o preço/pontos do primeiro resultado (x,y,w,h):
    # Comece usando algo como 900,800,350,140 e ajuste olhando screenshots salvos.
    "regiao_preco": (930, 800, 350, 140),
    # Opcional: clique em algum espaço em branco da página para garantir foco
    "foco": (1000, 830),
}

# Limite de pontos
LIMITE_PONTOS = 30000

# Datas (inclusive)
DATA_INICIO = dt.date(2025, 9, 25)
DATA_FIM    = dt.date(2025, 10, 13)

# Tempo de espera entre ações (ajuste se o site estiver mais lento)
WAIT_TYPE = 0.15       # tempo entre teclas
WAIT_AFTER_TYPE = 0.6  # tempo depois de digitar um campo
WAIT_RESULTS = 6.0     # tempo para carregar resultados
WAIT_AFTER_CLICK = 0.25

# -------------------- UTILS --------------------
def beep_ok():
    try:
        if platform.system() == "Windows":
            import winsound
            winsound.Beep(1000, 500)
        else:
            print("\a", end="", flush=True)  # bell
    except Exception:
        print("[BIP]")

def click_and_clear(x, y, backspaces=15):
    gui.click(x, y)
    time.sleep(WAIT_AFTER_CLICK)
    for _ in range(backspaces):
        gui.press('backspace')
        time.sleep(0.02)

def type_text(text, press_enter=False):
    gui.typewrite(text, interval=WAIT_TYPE)
    time.sleep(WAIT_AFTER_TYPE)
    if press_enter:
        gui.press('enter')
        time.sleep(WAIT_AFTER_CLICK)

def format_ddmmaaaa(d: dt.date) -> str:
    return d.strftime("%d%m%Y")

def daterange(start: dt.date, end: dt.date):
    cur = start
    while cur <= end:
        yield cur
        cur += dt.timedelta(days=1)

def ocr_text_from_region(region):
    """region: (x,y,w,h)"""
    x, y, w, h = region
    shot = gui.screenshot(region=(x, y, w, h))
    img = cv2.cvtColor(np.array(shot), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # binarização simples; ajuste o limiar se precisar
    _, bin_img = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    # OCR focado em dígitos/pontos/espacos
    config = '--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789 .'
    text = pytesseract.image_to_string(bin_img, config=config)
    return text, bin_img

def extrai_pontos(texto):
    """
    Tenta achar algo como '29.900', '30.000' etc.
    Retorna menor número encontrado (assumindo ser o menor preço visível).
    """
    # pega tokens numéricos tipo 12.345 ou 12345
    candidatos = re.findall(r'\d{1,3}(?:\.\d{3})+|\d+', texto)
    valores = []
    for c in candidatos:
        # remove pontos de milhar
        v = int(c.replace('.', ''))
        valores.append(v)
    return min(valores) if valores else None

def salvar_print_ok(prefixo, ida, origem, destino, bin_img):
    pasta = "prints_ok"
    os.makedirs(pasta, exist_ok=True)
    nome = f"{prefixo}_{origem}-{destino}_{ida.strftime('%Y-%m-%d')}.png"
    caminho = os.path.join(pasta, nome)
    cv2.imwrite(caminho, bin_img)
    return caminho

def preencher_origem_destino(origem, destino):
    # origem
    click_and_clear(*COORD["origem"])
    type_text(origem)
    time.sleep(0.4)
    gui.press('enter')
    time.sleep(0.4)

    # destino
    click_and_clear(*COORD["destino"])
    type_text(destino)
    time.sleep(0.4)
    gui.press('enter')
    time.sleep(0.4)

def preencher_datas(data_ida_str, data_volta_str=None):
    # data ida
    click_and_clear(*COORD["data_ida"], backspaces=8)
    type_text(data_ida_str)
    time.sleep(0.2)

    # data volta (se quiser repetir a mesma, como no seu pseudo)
    if data_volta_str:
        click_and_clear(*COORD["data_volta"], backspaces=8)
        type_text(data_volta_str)
        time.sleep(0.2)

def buscar():
    # garante foco e clica buscar
    gui.click(*COORD["foco"])
    time.sleep(0.1)
    gui.click(*COORD["buscar"])
    time.sleep(WAIT_RESULTS)

def rotina_busca(origem, destino, prefixo="ida"):
    for dia in daterange(DATA_INICIO, DATA_FIM):
        dstr = format_ddmmaaaa(dia)
        print(f"[{prefixo}] {origem}->{destino} | {dstr}")
        preencher_origem_destino(origem, destino)
        # replicando seu fluxo: preencher ida e volta com a mesma data
        preencher_datas(dstr, dstr)
        buscar()

        # OCR na região do preço/pontos
        texto, bin_img = ocr_text_from_region(COORD["regiao_preco"])
        pontos = extrai_pontos(texto)

        print(f"   OCR: {texto!r}  | pontos_detectados={pontos}")
        if pontos is not None and pontos <= LIMITE_PONTOS:
            beep_ok()
            path = salvar_print_ok(prefixo, dia, origem, destino, bin_img)
            print(f"   ✅ Achou ≤ {LIMITE_PONTOS} pontos! Screenshot salvo em: {path}")

# -------------------- MAIN --------------------
if __name__ == "__main__":
    print("Você tem 5 segundos para focar a janela do navegador no formulário de busca da Azul...")
    time.sleep(5)

    # 1) RECIFE -> GIG (como no seu pseudocódigo)
    rotina_busca("RECIFE", "GIG", prefixo="REC-GIG")

    # 2) GIG -> RECIFE


Você tem 5 segundos para focar a janela do navegador no formulário de busca da Azul...


KeyboardInterrupt: 

In [9]:
import time, re, platform, os
import datetime as dt
import pyautogui as gui
import numpy as np
import cv2
import pytesseract


# ========== CONFIG RÁPIDA ==========
# Se estiver no Windows e o Tesseract não estiver no PATH, aponte aqui:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Região onde aparece o preço/pontos do 1º resultado (x, y, w, h) -> AJUSTE SE PRECISAR
REGIAO_PRECO = (930, 800, 350, 140)

LIMITE_PONTOS = 30000
ESP_ACAO = 0.25        # pequena pausa entre cliques/teclas
ESP_RESULT = 6.0       # tempo pra carregar resultados após buscar

# Datas (inclusive)
d_ini = dt.date(2025, 9, 25)
d_fim = dt.date(2025, 10, 13)

# ========== FUNÇÕES MÍNIMAS (só o que é inevitável) ==========
def bip():
    try:
        if platform.system() == "Windows":
            import winsound; winsound.Beep(1000, 500)
        else:
            print("\a", end="", flush=True)
    except:
        print("[BIP]")

def ocr_pontos():
    x, y, w, h = REGIAO_PRECO
    shot = gui.screenshot(region=(x, y, w, h))
    img = cv2.cvtColor(np.array(shot), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bin_img = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    cfg = '--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789 .'
    txt = pytesseract.image_to_string(bin_img, config=cfg)
    # pega números tipo "29.900" ou "30000"
    nums = re.findall(r'\d{1,3}(?:\.\d{3})+|\d+', txt)
    vals = [int(n.replace('.', '')) for n in nums] if nums else []
    return (min(vals) if vals else None), txt

# ========== COMEÇAR ==========
print("Foque o navegador na página de busca da Azul. Começando em 5s...")
time.sleep(5)

# --------- BLOCO 1: RECIFE -> GIG (seguindo teu pseudo) ---------
# 500 400 | BACKSPACE
gui.click(500, 400); time.sleep(ESP_ACAO); gui.press('backspace'); time.sleep(ESP_ACAO)
# 813 400 | BACKSPACE
gui.click(813, 400); time.sleep(ESP_ACAO); gui.press('backspace'); time.sleep(ESP_ACAO)
# 500 400 | TYPE RECIFE | WAIT 3s | ENTER
gui.click(500, 400); time.sleep(ESP_ACAO)
gui.typewrite('RECIFE', interval=0.12); time.sleep(3); gui.press('enter'); time.sleep(ESP_ACAO)
# type GIG | WAIT 3s | ENTER (foco já deve estar no campo destino)
gui.typewrite('GIG', interval=0.12); time.sleep(3); gui.press('enter'); time.sleep(ESP_ACAO)

# FOR(25092025 ate 13102025) { ... }
d = d_ini
while d <= d_fim:
    dstr = d.strftime('%d%m%Y')
    print(f"[REC->GIG] Data: {dstr}")


    # 1020 400 | data volta | backspace 8x | digita ddmmaaaa (mesma data)
    gui.click(1020, 400); time.sleep(ESP_ACAO)
    for _ in range(8): gui.press('backspace'); time.sleep(0.02)
    gui.typewrite(dstr, interval=0.08); time.sleep(ESP_ACAO)

    # 1200 900 (buscar) | 1000 830 (focar/garantir) | espera resultados
    gui.click(1200, 900); time.sleep(ESP_ACAO)
    gui.click(1000, 830); time.sleep(ESP_RESULT)

    # scrap if 30.000 pontos ou menos -> bip
    pontos, bruto = ocr_pontos()
    print(f"   OCR: {bruto!r}  => pontos={pontos}")
    if pontos is not None and pontos <= LIMITE_PONTOS:
        bip()
        # (Opcional) salvar print do acerto:
        # gui.screenshot(f"REC-GIG_{d.isoformat()}.png")

    d += dt.timedelta(days=1)

# --------- BLOCO 2: GIG -> RECIFE (espelho do pseudo) ---------
# 500 400 | BACKSPACE
gui.click(500, 400); time.sleep(ESP_ACAO); gui.press('backspace'); time.sleep(ESP_ACAO)
# 813 400 | BACKSPACE
gui.click(813, 400); time.sleep(ESP_ACAO); gui.press('backspace'); time.sleep(ESP_ACAO)
# 500 400 | TYPE GIG | WAIT 3s | ENTER
gui.click(500, 400); time.sleep(ESP_ACAO)
gui.typewrite('GIG', interval=0.12); time.sleep(3); gui.press('enter'); time.sleep(ESP_ACAO)
# type RECIFE | WAIT 3s | ENTER
gui.typewrite('RECIFE', interval=0.12); time.sleep(3); gui.press('enter'); time.sleep(ESP_ACAO)

# FOR(25092025 ate 13102025) { ... }
d = d_ini
while d <= d_fim:
    dstr = d.strftime('%d%m%Y')
    print(f"[GIG->REC] Data: {dstr}")


    gui.click(1020, 400); time.sleep(ESP_ACAO)
    for _ in range(8): gui.press('backspace'); time.sleep(0.02)
    gui.typewrite(dstr, interval=0.08); time.sleep(ESP_ACAO)

    gui.click(1200, 900); time.sleep(ESP_ACAO)
    gui.click(1000, 830); time.sleep(ESP_RESULT)

    pontos, bruto = ocr_pontos()
    print(f"   OCR: {bruto!r}  => pontos={pontos}")
    if pontos is not None and pontos <= LIMITE_PONTOS:
        bip()
        # (Opcional) salvar print do01102025
        # gui.screenshot(f"GIG-REC_{d.isoformat()}.png")

    d += dt.timedelta(days=1)

print("Concluído.")


Foque o navegador na página de busca da Azul. Começando em 5s...
[REC->GIG] Data: 25092025
   OCR: '17\n'  => pontos=17
[REC->GIG] Data: 26092025
   OCR: ''  => pontos=None
[REC->GIG] Data: 27092025
   OCR: ''  => pontos=None
[REC->GIG] Data: 28092025
   OCR: ''  => pontos=None
[REC->GIG] Data: 29092025
   OCR: '430000\n128700\n'  => pontos=128700
[REC->GIG] Data: 30092025
   OCR: ''  => pontos=None
[REC->GIG] Data: 01102025


KeyboardInterrupt: 